In [15]:
!pip install sentence-transformers faiss-cpu pandas streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [16]:
from google.colab import files
uploaded = files.upload()  # Upload quotes.csv


Saving Quote_data.csv to Quote_data (1).csv


In [18]:
import pandas as pd

# Load and clean data
df = pd.read_csv("/content/Quote_data (1).csv")
df = df[['Quote', 'Author', 'revised_tags']].dropna().drop_duplicates()
df = df.reset_index(drop=True)


In [21]:
from sentence_transformers import SentenceTransformer

# ✅ Load pretrained model without training
model = SentenceTransformer('all-MiniLM-L6-v2')


In [24]:
# Reduce dataset to 2000 quotes for faster indexing
df = df.sample(2000, random_state=42).reset_index(drop=True)

quotes = df['Quote'].tolist()
quote_embeddings = model.encode(quotes, show_progress_bar=True, batch_size=32)
quote_embeddings = np.array(quote_embeddings).astype("float32")

index = faiss.IndexFlatL2(quote_embeddings.shape[1])
index.add(quote_embeddings)


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [25]:
import pickle

# Save FAISS index
faiss.write_index(index, "quote_index.faiss")

# Save quote metadata (quote, author, revised_tags)
with open("quote_metadata.pkl", "wb") as f:
    pickle.dump(df[['Quote', 'Author', 'revised_tags']], f)


In [26]:
import streamlit as st
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load FAISS index and metadata
index = faiss.read_index("quote_index.faiss")
with open("quote_metadata.pkl", "rb") as f:
    df = pickle.load(f)

# Search function
def search_quotes(query, top_k=5):
    query_vector = model.encode([query]).astype("float32")
    D, I = index.search(query_vector, top_k)
    results = []
    for idx in I[0]:
        row = df.iloc[idx]
        results.append({
            "quote": row['Quote'],
            "author": row['Author'],
            "tags": row['revised_tags']
        })
    return results

# Streamlit UI
st.title("🔍 Semantic Quote Finder")
query = st.text_input("Enter a theme, topic, or author (e.g., 'life', 'Einstein'):")

if query:
    results = search_quotes(query)
    for res in results:
        st.markdown(f"> *{res['quote']}* — **{res['author']}**")
        st.markdown(f"**Tags:** {res['tags']}")
        st.markdown("---")


2025-06-15 18:01:54.784 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-15 18:01:55.129 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-15 18:01:55.140 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-15 18:01:55.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-15 18:01:55.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-15 18:01:55.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-15 18:01:55.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-15 18:01:55.169 Session state does not 